In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
from functools import reduce

In [2]:
others_old_house = gpd.read_file('7.4개시도_주택노후도.geojson')
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')
# others_use_area = gpd.read_file('9.4개시도_용도지역도.geojson')
# others_business_area = gpd.read_file('10.4개시도_사업지구경계도.geojson')
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_security_lamp = pd.read_csv('22.4개시도_보안등현황.csv')
others_security_bell = gpd.read_file('23.4개시도_안전비상벨현황.geojson')
others_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
others_park =pd.read_csv('27.4개시도_공원현황.csv')

In [3]:
def point_police(data_df,others_police_area,col):
    df = data_df.copy()
    if "geometry" in df.columns:
        pass
    else:
        epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
        df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
        df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_result = gpd.sjoin(others_police_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_result_groupby = police_result.groupby("관할경찰서").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_result,police_result_groupby

관할 경찰서 경계

In [4]:
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_area.columns = ["관할경찰서","관할경찰청","geometry"]
others_police_area= others_police_area[['관할경찰서','geometry']]

In [9]:
others_old_house.columns = ["원천공간정보ID","건물명칭","동명칭","필지고유번호","구조명","용도명","승인일자","건물연한","건물노후도","geometry"]
others_old_house = others_old_house[["구조명","용도명","건물연한","건물노후도","geometry"]]


others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')[['BUSINESS_NM','COM_LC_NM','COM_MC_NM',
                'COM_SC_NM','KSIC_NM','SGG_NM',
       'ADB_EMD_NM','EMD_NM','ROAD_NM','geometry']]
others_alcohol.columns = ["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","표준산업분류명",
                         "시군구명","행정동명","법정동명","도로명","geometry"]

# others_use_area.columns = ["관리번호","별칭","비고","고시일자","원천오브젝트ID","원천시군구코드","geometry"]
others_security_lamp.columns = ["설치년도","설치주소","설치대수","lon","lat"]
others_security_bell.columns = ["설치목적","설치장소유형","설치위치","설치년도","geometry"]
others_cctv.columns = ["CCTV설치위치","CCTV설치목적","CCTV설치대수","CCTV설치년월","lat","lon"]

최종 결합 police_result copy

In [12]:
police_old_house = gpd.sjoin(others_police_area, others_old_house, how='left', op="intersects")

In [13]:
police_old_house_groupby = police_old_house.groupby("관할경찰서").agg({"건물연한":"mean","건물노후도":"mean"}).reset_index()

,관할경찰서,건물연한,건물노후도
0,마산동부경찰서,34.221704,3.520023
1,마산중부경찰서,27.137778,3.040000
2,서울강남경찰서,26.375468,3.038648


In [ ]:
police_result = pd.merge(police_result,police_old_house_groupby,how ='left',on = '관할경찰서')
police_result.head(3)

생활관련 시설 count

In [15]:
police_용도명 = police_old_house.groupby(["관할경찰서","용도명"]).agg({"구조명":"count"}).reset_index().rename(columns={"구조명":"cnt"})
police_생활관련 = police_용도명[(police_용도명["용도명"] == "제1종근린생활시설") |
(police_용도명["용도명"] == "제2종근린생활시설") |
(police_용도명["용도명"] == "근린생활시설") |
(police_용도명["용도명"] == "단독주택") |
(police_용도명["용도명"] == "공동주택")].groupby("관할경찰서").agg({"cnt":"sum"}).reset_index()

In [16]:
police_생활관련.columns = ['관할경찰서','생활관련시설']

8.4개시도_유흥업소현황.geojson

In [17]:
police_alcohol,police_alcohol_groupby = point_police(others_alcohol,others_police_area,"상호명")

In [37]:
police_alcohol_groupby.columns=['관할경찰서','유흥업소']

22.4개시도_보안등현황.csv

In [39]:
police_security_lamp, police_security_lamp_groupby = point_police(others_security_lamp, others_police_area,'geometry')

In [44]:
police_security_lamp_groupby = police_security_lamp.groupby('관할경찰서').agg({'설치대수':'sum'}).reset_index()

In [48]:
police_security_lamp_groupby.columns = ['관할경찰서','보안등']

In [49]:
# 동대문구, 양천구 데이터 없음
police_security_lamp_groupby.head()

,관할경찰서,보안등
0,마산동부경찰서,5647
1,마산중부경찰서,6821
2,서울강남경찰서,5029
3,서울강동경찰서,7500
4,서울강북경찰서,8431


23.4개시도_안전비상벨현황.geojson

In [57]:
police_security_bell,police_security_bell_groupby = point_police(others_security_bell,others_police_area,'geometry')

In [58]:
police_security_bell_groupby.columns = ['관할경찰서','안전비상벨']

'24.4개시도_CCTV설치현황.csv'

In [52]:
police_cctv,police_cctv_groupby = point_police(others_cctv,others_police_area,"geometry")

In [53]:
police_cctv_groupby = police_cctv.groupby('관할경찰서').agg({'CCTV설치대수':'sum'}).reset_index()

27.4개시도_공원현황.csv

In [25]:
police_park,police_park_groupby = point_police(others_park,others_police_area,"geometry")

In [55]:
police_park_groupby.columns = ['관할경찰서','공원']

In [62]:
merge_list=[police_old_house_groupby,police_생활관련,police_alcohol_groupby,police_security_lamp_groupby,
           police_security_bell_groupby,police_cctv_groupby,police_park_groupby] 
police_result = reduce(lambda left,right: pd.merge(left,right,how='outer',on = ['관할경찰서']),merge_list)
police_result.fillna(0,inplace=True)

In [72]:
police_result.head()

,관할경찰서,건물연한,건물노후도,생활관련시설,유흥업소,보안등,안전비상벨,CCTV설치대수,공원
0,마산동부경찰서,34.221704,3.520023,19177,164,5647.0,11.0,1218.0,65
1,마산중부경찰서,27.137778,3.040000,12071,88,6821.0,14.0,911.0,55
2,서울강남경찰서,26.375468,3.038648,11410,110,5029.0,649.0,3095.0,24
3,서울강동경찰서,26.086668,3.049578,17318,154,7500.0,2.0,0.0,72
4,서울강북경찰서,31.807006,3.248836,25777,128,8431.0,22.0,2299.0,41
